<a href="https://colab.research.google.com/github/405620294/iShape/blob/main/Cascade_4_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 409
# c 114
# w 105
# h 103
# b 26.1
age = 23
height = 179
weight = 98.7
chw_cm = 36.892
ww_cm = 42.291
hw_cm = 36.892
bmi = round(weight / ((height / 100) ** 2), 4)

In [ ]:
### user_info ###

user_info = {'age': age,
        'height': height,
        'weight': weight,
        'chest_width_cm': chw_cm,
        'waist_width_cm': ww_cm,
        'hip_width_cm': hw_cm,
        'bmi': bmi
}

In [ ]:
import pandas as pd
user_df = pd.DataFrame.from_dict(user_info,orient='index').T
user_df

,age,height,weight,chest_width_cm,waist_width_cm,hip_width_cm,bmi
0,23.0,179.0,98.7,36.892,42.291,36.892,30.8043


In [ ]:
import json
from scipy.special import boxcox1p, inv_boxcox1p
from sklearn.externals import joblib

# 預測胸圍
使用演算法；彈性網路(需做偏度調整與縮放)

In [ ]:
### adjust skewed to normal ###
# transpose user_info's keys to predict_df's columns
predict_df = pd.DataFrame.from_dict(user_info,orient='index').T

with open('/content/drive/Shareddrives/AI_Python/ishape/ai/step2/chest/ElasticNet/ElasticNet_lamda_2021-05-15_06:00:23.json', 'r') as f:
  chest_lamdas = json.load(f)

for i in predict_df.keys():
  predict_df[i]= boxcox1p(predict_df[i], chest_lamdas[i])

predict_df

,age,height,weight,chest_width_cm,waist_width_cm,hip_width_cm,bmi
0,8.969552,2.557766,4.881398,0.908787,1.87018,0.599433,1.049895


In [ ]:
### scaler ###
chest_scaler = joblib.load('/content/drive/Shareddrives/AI_Python/ishape/ai/step2/chest/ElasticNet/ElasticNet_MinMaxScaler_2021-05-15_06:00:23.h5')

predict_df = pd.DataFrame(chest_scaler.transform(predict_df), columns = predict_df.columns)

In [ ]:
model_chest = joblib.load('/content/drive/Shareddrives/AI_Python/ishape/ai/step2/chest/ElasticNet/ElasticNet_pre_chest_2021-05-15_06:00:23.h5')
pre = model_chest.predict(predict_df).round(4)
user_df['chest'] = inv_boxcox1p(pre, chest_lamdas['chest'])

# 預測腰圍
使用演算法；ridge (需做偏度調整與縮放)

In [ ]:
### adjust skewed to normal ###
predict_df = pd.DataFrame.from_dict(user_info,orient='index').T

with open('/content/drive/Shareddrives/AI_Python/ishape/ai/step2/waist/Ridge/Ridge_lamda_2021-05-15_05:23:01.json', 'r') as f:
  waist_lamdas = json.load(f)

for i in predict_df.keys():
  predict_df[i]= boxcox1p(predict_df[i], waist_lamdas[i])

In [ ]:
### scaler ###
waist_scaler = joblib.load('/content/drive/Shareddrives/AI_Python/ishape/ai/step2/waist/Ridge/Ridge_scaler_2021-05-15_05:23:01.h5')

predict_df = pd.DataFrame(waist_scaler.transform(predict_df), columns = predict_df.columns)

In [ ]:
model_waist = joblib.load('/content/drive/Shareddrives/AI_Python/ishape/ai/step2/waist/Ridge/Ridge_pre_waist_2021-05-15_05:23:01.h5')
pre = model_waist.predict(predict_df).round(4)
user_df['waist'] = inv_boxcox1p(pre, waist_lamdas['waist'])

# 預測臀圍
使用演算法；KNN(需做縮放)


In [ ]:
### scaler ###
predict_df = pd.DataFrame.from_dict(user_info,orient='index').T

hips_scaler = joblib.load('/content/drive/Shareddrives/AI_Python/ishape/ai/step2/hips/KNeighborsRegressor/KNeighborsRegressor_MinMaxScaler_2021-05-15_05:54:15.h5')

predict_df = pd.DataFrame(hips_scaler.transform(predict_df), columns = predict_df.columns)

In [ ]:
model_hips = joblib.load('/content/drive/Shareddrives/AI_Python/ishape/ai/step2/hips/KNeighborsRegressor/KNeighborsRegressor_pre_hips_2021-05-15_05:54:15.h5')
user_df['hips'] = model_hips.predict(predict_df).round(4)

In [ ]:
user_df

,age,height,weight,chest_width_cm,waist_width_cm,hip_width_cm,bmi,chest,waist,hips
0,23.0,179.0,98.7,36.892,42.291,36.892,30.8043,112.230737,120.565439,106.8286


# 預測體脂肪
使用演算法；knn (需做縮放)

In [ ]:
predict_df = user_df[['age', 'height', 'weight', 'chest', 'waist', 'hips', 'bmi']]

In [ ]:
### scaler ###
bfp_scaler = joblib.load('/content/drive/Shareddrives/AI_Python/ishape/ai/step3/KNeighborsRegressor/KNeighborsRegressor_MinMaxScaler_2021-05-15_08:58:11.h5')
predict_df = pd.DataFrame(bfp_scaler.transform(predict_df), columns = predict_df.columns)

In [ ]:
model_bfp = joblib.load('/content/drive/Shareddrives/AI_Python/ishape/ai/step3/KNeighborsRegressor/KNeighborsRegressor_pre_bodyfat_2021-05-15_08:58:11.h5')
user_df['pre_bfp'] = model_bfp.predict(predict_df).round(2)

In [ ]:
user_df

,age,height,weight,chest_width_cm,waist_width_cm,hip_width_cm,bmi,chest,waist,hips,pre_bfp
0,23.0,179.0,98.7,36.892,42.291,36.892,30.8043,112.230737,120.565439,106.8286,27.93


In [ ]:
### 分析 ###
standard_weight = round(( user_df['height'] - 80 ) * 0.7, 2) # 標準體重
wh_ratio = round(user_df['waist'] / user_df['hips'],2) # 腰臀比

chest = round(user_df['chest'].values[0], 2)
waist = round(user_df['waist'].values[0], 2)
hips = round(user_df['hips'].values[0], 2)

import datetime
now = datetime.datetime.now().strftime('%Y-%m-%d') # 當前時間(注意為伺服器時間)
print('當前時間', now)

當前時間 2021-05-17


# 使用者分析報告

In [ ]:
print('iShape分析報告')
print('-------------------')
print('檢測日期: ', now)
print('-------------------')

print('您的年齡 : ', user_df['age'].values[0])
print('您的身高 : ', user_df['height'].values[0], 'cm')
print('您的體重 : ', user_df['weight'].values[0], 'kg')

print('--------------------------------------')

print('您的標準體重 : ', standard_weight.values[0], 'kg')
print('--------------------------------------')

print('您的胸圍 : ', chest, 'cm')
print('--------------------------------------')

if waist > 90 :
  print('您的腰圍 : ', waist, 'cm')
  print('已超過衛福部規定男性健康腰圍')
  print('請注意內臟脂肪過高，心血管疾病、糖尿病、高血壓等風險')
else:
  print('您的腰圍 : ', waist, 'cm, 屬於健康範圍，請繼續保持')

print('--------------------------------------')

print('您的臀圍 : ', hips, 'cm')
print('--------------------------------------')

print('您的腰臀比 : ', wh_ratio.values[0], '%')
print('--------------------------------------')

bfp = user_df['pre_bfp'].values[0]
while True:
  print('您的體脂率 : ', bfp, '%')
  if bfp > 25:
    print('您的體脂率已超過25%, 屬於過胖, 請多做運動，減少精緻食物攝取')
  elif 15 < bfp <= 25:
    print('您的體脂率為健康範圍, 請繼續保持')
  elif bfp < 5:
    print('您的體脂率低於5%, 將影響身體機能, 請多多攝取蛋白質與堅果類')
  break

print('--------------------------------------')

iShape分析報告
-------------------
檢測日期:  2021-05-17
-------------------
您的年齡 :  23.0
您的身高 :  179.0 cm
您的體重 :  98.7 kg
--------------------------------------
您的標準體重 :  69.3 kg
--------------------------------------
您的胸圍 :  112.23 cm
--------------------------------------
您的腰圍 :  120.57 cm
已超過衛福部規定男性健康腰圍
請注意內臟脂肪過高，心血管疾病、糖尿病、高血壓等風險
--------------------------------------
您的臀圍 :  106.83 cm
--------------------------------------
您的腰臀比 :  1.13 %
--------------------------------------
您的體脂率 :  27.93 %
您的體脂率已超過25%, 屬於過胖, 請多做運動，減少精緻食物攝取
--------------------------------------
